In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import time
def getHtml(method, url, params= None, data= None, encoding= 'utf-8'):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36 Edg/86.0.622.69"}

    # headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
    # req = requests.request(req_type, url, data)
        headers = {'user-agent':str(UserAgent().random)}

        if method == 'get':
            req = requests.get(url, headers= headers, params= params)
        elif method == 'post':
            req = requests.post(url, headers= headers, data= data)
            
        # req=requests.get(url, headers= headers, params= params)
        if req.status_code == 200:
            print('requests -> 200')
            req.encoding = encoding
            return BeautifulSoup(req.text, 'html.parser')
        print('requests -> ', req.status_code)
        return None
    except:
        return None
    
def textClean(splitChar='', text='', loop=1, reRight=True, notReverse = True): #
    myIndex = 1 if notReverse else 0
    myLoop = 0
    myRange = text if notReverse else reversed(text)
    for i in myRange:
        if i == splitChar: 
            myLoop+=1
            if myLoop==loop: 
                if reRight:
                    return text[myIndex:] if notReverse else text[-myIndex:] # 由index至右
                return text[:myIndex] if notReverse else text[:-myIndex] # 由左至index
        myIndex+=1

In [2]:
url = "https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD&vs=list&pi=2"
soup = getHtml(method='get', url=url)

In [4]:
soup

In [72]:

reqPage = 1
reqKey = 'python'
reqMethod = 'key'
method = {
    'key':'N',
    'isbn':'ISBN',
    'au':'AU',
    'pu':'PU'
}[reqMethod]
# reqSort = ['price', 0]  # user request, init is none!
reqSort = None
sort = {'price': 'P','date': 'D',}[reqSort[0]] + ['D', 'A'][reqSort[1]] if reqSort != None else "SD"
resultURL = 'https://www.sanmin.com.tw/search/index?ct={}&K={}&ls={}&vs=List&pi={}&vs=list%20sbs'.format(method, reqKey, sort,reqPage) # 雙排
print(resultURL)

https://www.sanmin.com.tw/search/index?ct=N&K=python&ls=SD&vs=List&pi=1&vs=list%20sbs


In [ ]:
# # '並排'葉面, 靠北 沒日期作者資訊
# def sanmin(reqPage=1, getLastPage=1):
#     soup = getHtml(method='get', url='https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD&vs=list&pi=1&vs=grid')    
    
#     title = [textClean('.', i.text.strip()) for i in soup.find('ul', class_='GridView').findAll('h3')]
#     price = [i.text for i in soup.findAll('span', {'style':'color: red; font-size: 20px'})]

In [3]:
# new search
test=''
def sanmin(reqPage=1, getLastPage=1):  # 列表
#     soup = getHtml(method='get', url='https://www.sanmin.com.tw/search/index?ct=N&K=python&ls=SD&vs=List&pi=132&vs=list%20sbs')    
    soup = getHtml(method='get', url='https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD&vs=list%20sbs&pi={}'.format(reqPage))    
    def infoLoop(spanClass=""):
        return [i.text for i in soup.findAll('span', class_=spanClass)]
    soup = soup.find('div', id='parallel-list')
    if soup.find('div', class_='condition-parallel'):
        title = [textClean('.', i.text) for i in soup.findAll('h3')]
        img  = [textClean('/', i['original'], loop=2, notReverse=False) for i in soup.findAll('img', class_='lazyload')]  
        date = [textClean('：', i.text) for index, i in enumerate(soup.findAll('span', class_='pap')) if (index+1)%3==0]
        
        prodAu = infoLoop('ProdAu')
        prodPu = infoLoop('ProdPu')
        price = infoLoop('price')
        if len(price)!=len(title):  # 如果沒'優惠價'時, 取得'定價'價格, 並插入對應index.
            tmp = soup.findAll('div', class_='resultBooksLayout')
            for index, i in enumerate(tmp):
                if i.find('span', class_='price')==None:
                    price.insert(index, re.sub('\D', "", i.p.text.strip()))
                
        # --- result ---
        data = {
            'status':True,
            'info':[
                {
                    'img':img[i],
                    'title':title[i],
                    'price':price[i],
                    'date':date[i],
                    'author':prodAu[i],
                    'pub':prodPu[i],
                } for i in range(len(title))
            ],
#             'page':soup.find('a', class_='arrivel')['value']
        }
#         if getLastPage==1:  data['last'] =  soup.find('div', class_='PageLink').findAll('a')[-1]['value']
        return data
    
    return {
        'status':False,
        'info':[],
    }

In [35]:
sanmin(reqPage=131, getLastPage=11)

requests -> 200


{'status': True,
 'info': [{'img': '751/751237371.jpg',
   'title': 'MongoDB應用設計模式（簡體書）',
   'author': '(美)瑞克‧科普蘭\xa0',
   'pub': '中國電力出版社\xa0',
   'date': '2015/05/01',
   'price': '174'},
  {'img': '711/711536660.jpg',
   'title': '正則表達式經典實例（簡體書）',
   'author': '高瓦特斯\xa0',
   'pub': '人民郵電出版社\xa0',
   'date': '2014/10/01',
   'price': '443'},
  {'img': '986/986571266.jpg',
   'title': 'Raspberry Pi 輕鬆上手指南',
   'author': '陳建皓\xa0',
   'pub': '佳魁資訊\xa0',
   'date': '2014/06/27',
   'price': '378'},
  {'img': '711/711533186.jpg',
   'title': '火星人：Maya2014超級白金手冊(下)（簡體書）',
   'author': '火星時代\xa0',
   'pub': '人民郵電出版社\xa0',
   'date': '2013/11/01',
   'price': '857'},
  {'img': '730/730222516.jpg',
   'title': 'Linux下的MySQL數據庫編程(配光盤)（簡體書）',
   'author': '趙廷濤;劉冰\xa0',
   'pub': '清華大學出版社(大陸)\xa0',
   'date': '2010/06/01',
   'price': '286'},
  {'img': '711/711522832.jpg',
   'title': '正則表達式經典實例（簡體書）',
   'author': '(美)高瓦特斯(Goyvaerts J.);(美)利維森(Levithan S.)\xa0',
   'pub': '人民郵電出版社\xa0',
   'da

In [15]:
test = getHtml(method='get', url='https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD&vs=list&pi=%s' % '1')

requests -> 200


In [28]:
soup

<!DOCTYPE html>

<html>
<head>
<script>
        var ServerName = '';

    </script>
<!-- Google Tag Manager -->
<script>
        (function (w, d, s, l, i) {
            w[l] = w[l] || []; w[l].push({
                'gtm.start':
                    new Date().getTime(), event: 'gtm.js'
            }); var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                    'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
        })(window, document, 'script', 'sanminCrossDomain', 'GTM-KW5B62Q');
    </script>
<script>
        (function (w, d, s, l, i) {
            w[l] = w[l] || []; w[l].push({
                'gtm.start':
                    new Date().getTime(), event: 'gtm.js'
            }); var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                    'ht

In [29]:
reqPage = 1
url = 'https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD%20sbs&vs=list&pi={}'.format(reqPage)
# soup = getHtml(method='get', url=url)
print(url)

https://www.sanmin.com.tw/search/index?ct=K&K=python&ls=SD%20sbs&vs=list&pi=1


In [11]:
# old search
def b1(soup):
    data1 = []
    for i, info in enumerate(soup.findAll('div', class_="resultBooks")):
        AuPuDe = info.findAll('span', class_="pap")
        ImIn = info.find('a')
        data1.append(
            {
    #             "im": info.div.a.img["original"][37:],
                "im": ImIn.img["original"][37:],
                "in": textClean('/', ImIn['href'], notReverse=False),
                "de": textClean('：', AuPuDe[2].text),
                "ti": textClean(".", info.h3.text),
                "au": AuPuDe[0].span.a.text,
                "pu": AuPuDe[1].span.a.text,
            }
        )
    print({
        'status':True,
        'info':data1
    })


In [6]:
    def b2(soup):
        def infoLoop(spanClass=""):
            return [i.text for i in soup.findAll('span', class_=spanClass)]
        soup = soup.find('div', id='parallel-list')
        if soup.find('div', class_='condition-parallel'):
            title = [textClean('.', i.text) for i in soup.findAll('h3')]
            img  = [textClean('/', i['original'], loop=2, notReverse=False) for i in soup.findAll('img', class_='lazyload')]  
            date = [textClean('：', i.text) for index, i in enumerate(soup.findAll('span', class_='pap')) if (index+1)%3==0]

            prodAu = infoLoop('ProdAu')
            prodPu = infoLoop('ProdPu')
            price = infoLoop('price')
            if len(price)!=len(title):  # 如果沒'優惠價'時, 取得'定價'價格, 並插入對應index.
                tmp = soup.findAll('div', class_='resultBooksLayout')
                for index, i in enumerate(tmp):
                    if i.find('span', class_='price')==None:
                        price.insert(index, re.sub('\D', "", i.p.text.strip()))

            # --- result ---
            data = {
                'status':True,
                'info':[
                    {
                        'img':img[i],
                        'title':title[i],
                        'author':prodAu[i],
                        'pub':prodPu[i],
                        'date':date[i],
                        'price':price[i],
                    } for i in range(len(title))
                ],
#             'page':soup.find('a', class_='arrivel')['value']
            }
            print(data)
        else:
            print('error')


In [14]:
start = time.time()
# b1(soup)
b2(soup)
print(time.time()-start)

0.015000343322753906


In [87]:
# 邏輯插入
start = time.time()

oldPrice = price
price = []
if len(title)!=len(price):
    priceNone=[]
    tmp = soup.findAll('div', class_='resultBooksLayout')
    for index, i in enumerate(tmp):
        if i.find('span', class_='price')==None:
            print('None price : ',index)
            priceNone.append(index)

    price = oldPrice[:priceNone[0]]
    for index, i in enumerate(priceNone):
        newPrice = re.sub('\D', "", tmp[i].p.text.strip())
        price.append(newPrice)  
        if index+1 != priceNone[index+1]: # 
            if (index+1)<len(priceNone):
                tmp = oldPrice[i:priceNone[index+1]]
            else:
                tmp = oldPrice[i:]
            print('my tmp List : ', tmp)
            price.extend(tmp)


print(time.time()-start)


None price :  2
init tmpList :  ['412', '517', '定價：174元', '443', '378', '857', '286', '352', '653', '585']
my tmp List :  ['443', '378', '857', '286', '352', '653', '585']
0.03698229789733887


In [104]:
# 似乎比上面邏輯插入還慢喔, 有時間再思考上面程式碼
start = time.time()
tmp = soup.findAll('div', class_='resultBooksLayout')
for i in priceNone:
    price.insert(i, re.sub('\D', "", tmp[i].p.text.strip()))
print(time.time()-start)

0.05882000923156738


In [ ]:
# index search
def sanmin(soup, infoData):
     # --- Price ---  (簡單暴力法, 待修改)
    str1 = soup.find('script', {'type':'application/ld+json'}).contents[0].strip()
    data = json.loads(str1)
    data['offers']['price']
    # --- image URL ---
    # 書封面 cdnec.sanmin.com.tw/product_images/
    # other cdnec.sanmin.com.tw/tryread/big/
    tmp = soup.find('div', class_='gallery-top').findAll('div', class_='swiper-slide')   # 大圖示欄
    imgURL = [tmp[0].img['src'][37:]]  # first img 
    imgURL.extend([i.img['src'][40:] for i in tmp[1:]])  # other img
    # --- Information ---
    tag = {'書名':'title','ISBN':'isbn','頁數':'page','出版社':'pub','作者':'author', '出版日期':'date'}
    findInfo = soup.find('meta', {'name':'description'})['content']
    for i in findInfo.split('，'):  # book info處理
        tmp =  i.split('：')
        if len(tmp)>2: tmp = [tmp[0], textClean(splitChar='：', text=i, loop=1)] # 標題, 內容(防止內容出現':'有問題)
        key = tag[tmp[0]] if (tmp[0] in tag) else None # 確標籤有在需求(tag)裡
        if key: infoData[key] = tmp[1]  # 資料內容
    # --- Intr ---
    # 標籤 ---
    tag = {
        "商品簡介": 'intr',
        "作者簡介": 'author',
        "目次": 'contents',
    }
    tabTitle_find = soup.findAll('div', class_='tab2')
    intrData = [{"title": None, "content": None} for i in range(len(tabTitle_find))]
    for indexI, i in enumerate(tabTitle_find):
        tmp = i.text.strip()  # ex: 商品簡介
        intrData[indexI]['title'] = tag[tmp] if (tmp in tag) else None  # ex: intr, else-> None 
    # 內容 ---    
    contents_find = soup.findAll('div', class_='productContent')
    for indexI, i in enumerate(contents_find):
        word = ''
        for j in i:
            tmp = j.string
            # if text is None -> loop data get string then join '\n' 
            word += f'\n{tmp}\n' if tmp!=None else "\n".join([k.string for k in j if k.string!=None])+'\n'
        intrData[indexI]['content'] = word
    return {
        'imgURL': imgURL,
        'info': infoData,
        'intr': intrData,
    }


In [2]:
"https://www.sanmin.com.tw/search/index?ct=N&K=%s&ls=SD&vs=List&pi=132&vs=list%20sbs"%"dd"

'https://www.sanmin.com.tw/search/index?ct=N&K=dd&ls=SD&vs=List&pi=132&vs=list20sbs'